In [1]:
from huggingface_hub import login
login()

In [2]:
import warnings
warnings.filterwarnings("ignore")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import spacy
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
try:
    nlp = spacy.load("en_core_web_sm")
except:
    import os
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

In [4]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
peft_model_id = "devanshisingh/Llama-3.1-8B-isc"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [7]:
model = PeftModel.from_pretrained(base_model, peft_model_id)

ValueError: Can't find 'adapter_config.json' at 'devanshisingh/Llama-3.1-8B-isc'

In [1]:
def generate_responses(prompt, num_samples=3, max_new_tokens=200): 
    responses = []
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    for _ in range(num_samples):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,  
                temperature=0.8,
                top_p=0.9,
                repetition_penalty=1.2
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if response.startswith(prompt):
            response = response[len(prompt):].strip()
        responses.append(response)
    
    return responses

In [2]:
def get_sentences(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if len(sent) > 3]
    return sentences

In [3]:
def compute_bertscore(sentences, reference_samples):
    """
    Compute BERTScore between sentences and reference samples.
    """
    from selfcheckgpt.modeling_selfcheck import SelfCheckBERTScore
    
    bertscore = SelfCheckBERTScore()
    scores = bertscore.predict(sentences, reference_samples)
    return scores

In [4]:
def compute_mqag(sentences, main_text, reference_samples, device="cuda"):
    from selfcheckgpt.modeling_selfcheck import SelfCheckMQAG
    
    mqag = SelfCheckMQAG(device=device)
    scores = mqag.predict(
        sentences,
        main_text,
        reference_samples,
        num_questions_per_sent=3,  # Reduced for speed
        scoring_method='bayes_with_alpha',
        beta1=0.95, beta2=0.95,
    )
    return scores

In [5]:
def run_selfcheck(prompt, use_bertscore=True, use_mqag=True, num_samples=3):
    print(f"Generating {num_samples} responses for: {prompt}")
    main_response = generate_responses(prompt, num_samples=1)[0]
    print(f"\nMain response:\n{main_response}\n")
    reference_samples = generate_responses(prompt, num_samples=num_samples)
    print(f"Generated {len(reference_samples)} reference samples.")
    sentences = get_sentences(main_response)
    print(f"Analyzing {len(sentences)} sentences...")
    results = {
        "prompt": prompt,
        "main_response": main_response,
        "reference_samples": reference_samples,
        "sentences": sentences,
        "bertscore": None,
        "mqag": None
    }
    
    if use_bertscore:
        print("Computing BERTScore...")
        bertscore = compute_bertscore(sentences, reference_samples)
        results["bertscore"] = bertscore
    
    if use_mqag:
        print("Computing MQAG scores (this may take a while)...")
        mqag_scores = compute_mqag(sentences, main_response, reference_samples)
        results["mqag"] = mqag_scores
    
    hallucinations = []
    if use_bertscore and use_mqag:
        for i, (sentence, bert_score, mqag_score) in enumerate(zip(sentences, results["bertscore"], results["mqag"])):
            # Combined score
            combined_score = (bert_score + mqag_score) / 2
            if combined_score < 0.5:  # Threshold for hallucination
                hallucinations.append({
                    "sentence": sentence,
                    "bertscore": bert_score,
                    "mqag": mqag_score,
                    "combined": combined_score
                })
    elif use_bertscore:
        for i, (sentence, score) in enumerate(zip(sentences, results["bertscore"])):
            if score < 0.5:
                hallucinations.append({
                    "sentence": sentence,
                    "bertscore": score
                })
    elif use_mqag:
        for i, (sentence, score) in enumerate(zip(sentences, results["mqag"])):
            if score < 0.5:
                hallucinations.append({
                    "sentence": sentence,
                    "mqag": score
                })
    
    results["hallucinations"] = hallucinations
    return results

In [6]:
test_prompts = [
    "What is the capital of France?",
    "Explain the theory of relativity.",
    "Describe the history and cultural significance of the planet Nibiru.",
    "Who was the first human to land on Mars?",
    "What are the health benefits of drinking water?"
]

In [9]:
example_result = run_selfcheck(
    test_prompts[0], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: What is the capital of France?


NameError: name 'tokenizer' is not defined

In [10]:
example_result = run_selfcheck(
    test_prompts[1], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: Explain the theory of relativity.


NameError: name 'tokenizer' is not defined

In [11]:
example_result = run_selfcheck(
    test_prompts[2], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: Describe the history and cultural significance of the planet Nibiru.


NameError: name 'tokenizer' is not defined

In [12]:
example_result = run_selfcheck(
    test_prompts[3], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: Who was the first human to land on Mars?


NameError: name 'tokenizer' is not defined

In [13]:
example_result = run_selfcheck(
    test_prompts[4], 
    use_bertscore=True,
    use_mqag=True,
    num_samples=3
)

# Print the results
print("\nFinal results:")
print("Hallucinations found:", len(example_result["hallucinations"]))
for h in example_result["hallucinations"]:
    print(f"- {h['sentence']}")
    if "bertscore" in h:
        print(f"  BERTScore: {h['bertscore']:.4f}")
    if "mqag" in h:
        print(f"  MQAG Score: {h['mqag']:.4f}")
    if "combined" in h:
        print(f"  Combined Score: {h['combined']:.4f}")

Generating 3 responses for: What are the health benefits of drinking water?


NameError: name 'tokenizer' is not defined